In [1]:
## 사용 라이브러리 호출
import pandas as pd
from urllib.parse import quote

In [2]:
## 파라미터 예시 1

target = "name,time,value"
table = "home"
name = "'TAG-pressure','TAG-dewPoint'"
start = quote("2016-01-01 14:00:00")
end = quote("2017-01-01 14:00:10")

In [3]:
## 파라미터 예시 2

target = "name,time,value"
table = "home"
name = "'TAG-dewPoint'"
start = quote("'2016-01-01 14:00:00'")
end = quote("'2017-01-01 14:00:10'")

In [4]:
# 파라미터 예시 3 

target = "name,time,value"
table = "home"
name = "'TAG-dewPoint'"
start = quote("TO_DATE('2016-01-01 14:00:00.000000000', 'YYYY-MM-DD HH24:MI:SS.mmmuuunnn')")
end = quote("TO_DATE('2016-01-01 14:00:18.999999999', 'YYYY-MM-DD HH24:MI:SS.mmmuuunnn')")

In [5]:
# 파라미터 예시 4

target = "name,time,value"
table = "home"
name = "'TAG-pressure','TAG-dewPoint'"
start = "1451624400000000000"
end = "1451624407000000000"

## GET_TAG_NAME
* Machbase Neo 내부 사용자가 원하는 Tag Table 의 Tag Name 리스트를 호출
* Tag Name 은 Column name 을 의미
* 파라미터 
  * table : Tag Name 리스트를 호출할 table 이름 ex) ‘home’


In [6]:
# 파라미터 설정
table = 'home'

# Tag name 데이터 로드
df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}')

# List 형식으로 변환
df = df.values.reshape(-1).tolist()

df

['TAG-Barn [kW]',
 'TAG-Dishwasher [kW]',
 'TAG-Fridge [kW]',
 'TAG-Furnace 1 [kW]',
 'TAG-Furnace 2 [kW]',
 'TAG-Garage door [kW]',
 'TAG-Home office [kW]',
 'TAG-House overall [kW]',
 'TAG-Kitchen 12 [kW]',
 'TAG-Kitchen 14 [kW]',
 'TAG-Kitchen 38 [kW]',
 'TAG-Living room [kW]',
 'TAG-Microwave [kW]',
 'TAG-Solar [kW]',
 'TAG-Well [kW]',
 'TAG-Wine cellar [kW]',
 'TAG-apparentTemperature',
 'TAG-dewPoint',
 'TAG-gen [kW]',
 'TAG-humidity',
 'TAG-precipIntensity',
 'TAG-precipProbability',
 'TAG-pressure',
 'TAG-temperature',
 'TAG-use [kW]',
 'TAG-visibility',
 'TAG-windBearing',
 'TAG-windSpeed']

## Select-Rawdata
* Machbase Neo 내부 사용자가 원하는 Tag Table 의 Raw data 를 호출
* 파라미터
   * table : Tag Name 리스트를 호출할 Tag Table 이름
   * name : Tag Table 중 호출할 Tag Name
   * start : Tag Table 에서 Raw data 를 가져올 시작 시간
   * end : Tag Table 에서 Raw data 를 가져올 시작 시간
   * limit : 가져올 Raw data 의 개수 제한 (default = None)


In [7]:
# 데이터 로드 
df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start}&end={end}')

# 같은 시간대 별 데이터로 전환
df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()

df

NAME,TIME,TAG-dewPoint,TAG-pressure
0,2016-01-01 14:00:00,24.4,1016.91
1,2016-01-01 14:00:01,24.4,1016.91
2,2016-01-01 14:00:02,24.4,1016.91
3,2016-01-01 14:00:03,24.4,1016.91
4,2016-01-01 14:00:04,24.4,1016.91
5,2016-01-01 14:00:05,24.4,1016.91
6,2016-01-01 14:00:06,24.4,1016.91
7,2016-01-01 14:00:07,24.4,1016.91


## Select-Rollup

* Machbase Neo 내부 사용자가 원하는 Tag Table 의 Rollup Table 호출
* Rollup Table 원본 Tag Table 의 통계값들로 구성 
* 파라미터
  * table : Tag Name 리스트를 호출할 Tag Table 이름
  * name : Tag Table 중 호출할 Tag Name
  * start : Tag Table 에서 Raw data 를 가져올 시작 시간
  * end : Tag Table 에서 Raw data 를 가져올 시작 시간
  * func : Tag Table 에서 호출할 통계값 -> min, max, avg, count, sum, sumsq
  * timeunit : Tag Table 에서 얻어올 시간 통계의 종류 선택 -> sec, min, hour

In [8]:
func = 'avg'
timeunit = 'min'

# 데이터 로드 
df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rollup.tql?timeunit={timeunit}&table={table}&name={name}&func={func}&start={start}&end={end}')

# 같은 시간대 별 데이터로 전환
df = df.pivot_table(index='mtime', columns='name', values='avg(value)', aggfunc='first').reset_index()

df

name,mtime,TAG-dewPoint,TAG-pressure
0,2016-01-01 14:00:00,24.383333,1016.888


## Select-Scale

* Machbase Neo 내부 사용자가 원하는 Tag Table 의 Min, Max 값 호출
* 파라미터
  * table : Tag Name 리스트를 호출할 Tag Table 이름
  * name : Tag Table 중 호출할 Tag Name
  * start : Tag Table 에서 Raw data 를 가져올 시작 시간
  * end : Tag Table 에서 Raw data 를 가져올 시작 시간

In [9]:
# min max 데이터 로드
df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-scale.tql?table={table}&name={name}&start={start}&end={end}')

## Min , Max values 설정 
Min = df.iloc[:,1:-1].T
Max = df.iloc[:,2:].T

print(Min, Max)

        0        1
min  24.4  1016.91         0        1
max  24.4  1016.91
